In [1]:
import numpy as np

def process_tile(tile):
    k, *v = tile.split('\n')
    return int(k[5:9]), np.array([list(row) for row in v])

In [2]:
with open('input') as f:
    tiles = {
        k: {
            'tile': v,
            'edges': set(),
            'neighbours': set(),
        }
        for k, v in [
            process_tile(line.strip())
            for line in f.read().split('\n\n')
            if line != ''
        ]
    }

In [3]:
for tile_id, tile_dict in tiles.items():
    tile = tile_dict['tile']
    tile_dict['edges'] = {
        ''.join(tile[0]),
        ''.join(tile[-1]),
        ''.join(row[0] for row in tile),
        ''.join(row[-1] for row in tile),
        ''.join(tile[0])[::-1],
        ''.join(tile[-1])[::-1],
        ''.join(row[0] for row in tile)[::-1],
        ''.join(row[-1] for row in tile)[::-1],
    }

In [4]:
import itertools

for (tile_1_id, tile_1_dict), (tile_2_id, tile_2_dict) in itertools.combinations(tiles.items(), 2):
    tile_1_edges = tile_1_dict['edges']
    tile_2_edges = tile_2_dict['edges']
    for edge_1, edge_2 in itertools.product(tile_1_edges, tile_2_edges):
        if edge_1 == edge_2:
            tile_1_dict['neighbours'].add(tile_2_id)
            tile_2_dict['neighbours'].add(tile_1_id)
            
corners = {k: v for k, v in tiles.items() if len(v['neighbours']) == 2}
edges = {k: v for k, v in tiles.items() if len(v['neighbours']) == 3}
middles = {k: v for k, v in tiles.items() if len(v['neighbours']) == 4}

In [5]:
import math
print("Part 1:")
print(math.prod(corners))

Part 1:
18482479935793


In [6]:
def place_tiles():
    corner_tile_id = list(corners).pop()
    grid = {
        (0, 0): corner_tile_id,
    }
    pos = (0, 1)
    next_tile_id = list(tiles[corner_tile_id]['neighbours'])[0]
    grid[pos] = next_tile_id
    
    previous_tile_id = corner_tile_id
    current_tile_id = next_tile_id
    
    deltas = ((0, 1), (1, 0), (0, -1), (-1, 0))
    # do edges from top-left down
    for delta in deltas:
        while True:
            pos = (pos[0] + delta[0], pos[1] + delta[1])
            current_tile_neighbours = tiles[current_tile_id]['neighbours']
            next_tile_ids = list(current_tile_neighbours - {previous_tile_id} - set(middles.keys()))
            assert len(next_tile_ids) == 1
            next_tile_id = next_tile_ids[0]
            grid[pos] = next_tile_id
            previous_tile_id = current_tile_id
            current_tile_id = next_tile_id
            if current_tile_id in corners:
                break
                
    # fill in middles, from top left down in columns
    for y in range(1, 11):
        for x in range(1, 11):
            neighbours = {grid[(x - 1, y)], grid[(x, y - 1)]}
            tile_ids = [
                tile_id
                for tile_id, tile in middles.items()
                if tile['neighbours'] >= neighbours and tile_id not in set(grid.values())
            ]
            tile_id = tile_ids[0]
            grid[(x, y)] = tile_id
    
    return grid

In [7]:
grid = place_tiles()
print()
print("grid:")
print()
for y in range(12):
    for x in range(12):
        if (x, y) in grid:
            print(grid[(x, y)], end=' ')
        else:
            print('    ', end=' ')
    print()


grid:

1699 2803 1889 2297 2011 3851 1741 2467 1307 3089 1999 1433 
3251 3727 3407 2269 3209 1913 1831 1973 1153 2797 3257 1567 
3613 2851 1193 1657 3907 1487 3343 3637 2957 3821 1493 3217 
3739 2239 2003 2459 2281 1277 1201 1697 2903 3733 1163 2273 
1637 1279 2551 1291 2203 2423 2707 2693 1453 2477 3767 2767 
2729 1409 2029 3923 2437 1811 2647 2663 1987 3347 2141 1621 
3023 1259 3329 2237 3373 2377 1931 3049 3929 3499 1997 3863 
3673 2927 1787 1733 3911 1301 3803 2897 2969 3001 3761 1721 
1447 1327 1879 2383 2333 1747 2557 3449 1871 2689 3877 1873 
1663 1123 1181 2789 2027 1373 3323 3779 2083 3769 3529 3019 
1321 1549 2441 2687 2053 1907 1571 2801 1051 2819 2579 3719 
3229 1399 1063 2659 3061 1097 2243 1667 1091 3547 1609 2351 


In [8]:
def rotate_tile_to_match_below(x, y):
    this_tile = tiles[grid[(x, y)]]
    tile_below = tiles[grid[(x, y + 1)]]
    matching_edges = this_tile['edges'] & tile_below['edges']
    for i in range(4):
        bottom_edge = ''.join(this_tile['tile'][-1])
        if bottom_edge in matching_edges:
            return
        this_tile['tile'] = np.rot90(this_tile['tile'])

In [9]:
def flip_tile_to_align_left(x, y):
    this_tile = tiles[grid[(x, y)]]
    left_tile = tiles[grid[(x - 1, y)]]
    left_tile_right_edge = ''.join(row[-1] for row in left_tile['tile'])
    this_tile_left_edge = ''.join(row[0] for row in this_tile['tile'])
    if this_tile_left_edge != left_tile_right_edge:
        this_tile['tile'] = np.fliplr(this_tile['tile'])

In [10]:
def rotate_tile_to_match_above(x, y):
    this_tile = tiles[grid[(x, y)]]
    tile_above = tiles[grid[(x, y - 1)]]
    matching_edges = this_tile['edges'] & tile_above['edges']
    for i in range(4):
        top_edge = ''.join(this_tile['tile'][0])
        if top_edge in matching_edges:
            return
        this_tile['tile'] = np.rot90(this_tile['tile'])
    raise ValueError(f'rotate_tile_to_match_above: {x}, {y}')

In [11]:
def flip_tile_to_align_above(x, y):
    this_tile = tiles[grid[(x, y)]]
    tile_above = tiles[grid[(x, y - 1)]]
    above_bottom_edge = ''.join(tile_above['tile'][-1])
    this_top_edge = ''.join(this_tile['tile'][0])
    if this_top_edge != above_bottom_edge:
        this_tile['tile'] = np.fliplr(this_tile['tile'])

In [12]:
def assemble_image():
    x, y = (0, 0)
    # make sure (0,0) has its common edge facing downwards
    rotate_tile_to_match_below(x, y)
    # rotate and flip each following tile on the top row
    for x in range(1, 12):
        rotate_tile_to_match_below(x, y)
        flip_tile_to_align_left(x, y)
    # rotate and flip all remaining tiles to match the tile above them
    for y in range(1, 12):
        for x in range(12):
            rotate_tile_to_match_above(x, y)
            flip_tile_to_align_above(x, y)

In [13]:
def print_grid():
    for y in range(12):
        print(y)
        for row in range(10):
            for x in range(10):
                print(''.join(tiles[grid[(x, y)]]['tile'][row]).replace('#', str(x)), end=' ')
            print()
        print()

In [14]:
assemble_image()

In [15]:
image = np.empty((120, 120), dtype=str)
for pos, tile_id in grid.items():
    x, y = pos
    x0 = 8 * x
    y0 = 8 * y
    x1 = 8 * (x + 1)
    y1 = 8 * (y + 1)
    image[y0:y1, x0:x1] = tiles[tile_id]['tile'][1:-1,1:-1]

In [16]:
sea_monster = set()
sea_monster |= {(0, i) for i, c in enumerate('                  # ') if c == '#'}
sea_monster |= {(1, i) for i, c in enumerate('#    ##    ##    ###') if c == '#'}
sea_monster |= {(2, i) for i, c in enumerate(' #  #  #  #  #  #   ') if c == '#'}

In [17]:
def matches_sea_monster(x, y):
    return all(image[y + smy, x + smx] == '#' for smy, smx in sea_monster)

In [18]:
sea_monster_width = 20
sea_monster_height = 3
sea_monster_positions = set()

for i in range(2):
    for j in range(4):
        for x in range(image.shape[1] - sea_monster_width):
            for y in range(image.shape[0] - sea_monster_height):
                if matches_sea_monster(x, y):
                    for row in image[y:y+3,x:x+20]:
                        sea_monster_positions.add((x, y, i, j))
        image = np.rot90(image)
    image = np.fliplr(image)

In [19]:
for i in range(2):
    for j in range(4):
        for x in range(image.shape[1] - sea_monster_width):
            for y in range(image.shape[0] - sea_monster_height):
                if (x, y, i, j) in sea_monster_positions:
                    for smy, smx in sea_monster:
                        image[y + smy, x + smx] = 'O'
        image = np.rot90(image)
    image = np.fliplr(image)

In [20]:
print("Part 2:")
print(sum(''.join(row).count('#') for row in image))

Part 2:
2118
